In [4]:
print("ok")

ok


In [74]:
%pwd

'c:\\'

In [39]:
import os
os.chdir('C:/LLM/Labour-Law-chatbot/Data')


In [2]:
%pwd

'C:\\LLM\\Labour-Law-chatbot\\Data'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#Extract Data from the PDF File
os.chdir('C:/LLM/Labour-Law-chatbot')

def load_pdf_file(folder_name):
    loader = DirectoryLoader(folder_name, glob="*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents


In [5]:
extracted_data = load_pdf_file('Data')
print(f"{len(extracted_data)} PDFs loaded.")



197 PDFs loaded.


In [118]:
#extracted_data

In [6]:
#Split the Data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap =20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Length of text Chunks", len(text_chunks))

Length of text Chunks 888


In [15]:
#text_chunks

In [8]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings



In [9]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

C:\Users\krajk\AppData\Local\Temp\ipykernel_14064\3511447904.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")
c:\Users\krajk\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [39]:
#query_result

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")



None


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec 
import os

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name ="labourlawbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)



In [43]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GROQ_API_KEY"] = "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"


In [44]:
#Embed each chunk and upsert the embeddings into the Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding= embeddings,

)

In [45]:
#Load Existing index 

from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into your Pinecone index.

docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding= embeddings
)



In [46]:
docsearch

In [47]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs={"k":3})

In [48]:
retriever_docs = retriever.invoke("What is Labour Law?")

In [49]:
retriever_docs

[Document(id='da3efd69-2083-4435-9d1b-8c9c2152673a', metadata={'creationdate': '2012-10-08T10:05:47+02:00', 'creator': 'convertonlinefree.com', 'moddate': '2012-10-08T13:38:03+05:30', 'page': 2.0, 'page_label': '3', 'producer': 'convertonlinefree.com', 'source': 'Data\\Labour Act.pdf', 'total_pages': 197.0}, page_content='3 \n \n1. INTRODUCTION \nLabour law  also known as employment law  is the body of laws, administrative rulings, and \nprecedents which address the legal rights of, and restrictions on, working people and their \norganizations. As such, it mediates many aspects of the relationship between trade unions, \nemployers and employees. In other words, Labour law defines the rights and obligations as workers, \nunion members and employers in the workplace.  Generally, labour law covers:'),
 Document(id='c886fec2-17b8-4bb4-8452-a739eec80e05', metadata={'creationdate': '2012-10-08T10:05:47+02:00', 'creator': 'convertonlinefree.com', 'moddate': '2012-10-08T13:38:03+05:30', 'page'

In [55]:
# Load GROQ LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key="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",
    model_name="llama3-70b-8192",  # or llama2-70b-4096 / gemma-7b-it
    temperature=0.4,
    max_tokens=500,
)

In [56]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Keep the answer concise (max 3 sentences).\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)


In [57]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [61]:
response = rag_chain.invoke({"input": "what is nose?"})
print(response["answer"])

I don't know. The provided context is about registered trade unions, and there is no mention of "nose".
